In [14]:
import time
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

class BetmanCrawler:
    def __init__(self):
        self.driver = None
        self.results = []
        
    def setup_driver(self):
        """크롬 드라이버 설정"""
        chrome_options = Options()
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--window-size=1280,800')
        chrome_options.add_argument('--disable-blink-features=AutomationControlled')
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)
        # 헤드리스 모드 비활성화 (디버깅용)
        # chrome_options.add_argument('--headless')
        
        # User-Agent 설정
        chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
        
        # ChromeDriver 자동 설치 및 설정
        service = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=service, options=chrome_options)
        
        # 자동화 탐지 우회
        self.driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        
        print("크롬 드라이버 설정 완료")
        
    def navigate_to_page(self):
        """베트맨 페이지로 이동"""
        url = "https://www.betman.co.kr/main/mainPage/gamebuy/closedGameSlipIFR.do?gmId=G101&gmTs=250053&gameDivCd=C&isIFR=Y"
        
        try:
            self.driver.get(url)
            print(f"페이지 로딩 중: {url}")
            
            # 페이지 로딩 대기
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
            
            time.sleep(2)
            print("페이지 로딩 완료")
            
        except TimeoutException:
            print("페이지 로딩 시간 초과")
            raise
    
    def click_dropdown_to_open(self):
        """드롭다운을 클릭해서 옵션 목록을 표시"""
        try:
            # 드롭다운 요소 찾기
            dropdown_selectors = [
                "select[name='selectBoxGameRnd']",
                "#selectBoxGameRnd",
                ".selectBoxGameRnd",
                "select:first-of-type"
            ]
            
            dropdown_element = None
            for selector in dropdown_selectors:
                try:
                    WebDriverWait(self.driver, 3).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, selector))
                    )
                    dropdown_element = self.driver.find_element(By.CSS_SELECTOR, selector)
                    print(f"드롭다운 발견: {selector}")
                    break
                except:
                    continue
            
            if not dropdown_element:
                print("드롭다운 요소를 찾을 수 없습니다.")
                return False
            
            # 드롭다운 위치로 스크롤
            self.driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", dropdown_element)
            time.sleep(1)
            
            # 드롭다운 클릭해서 옵션 목록 열기
            try:
                # 먼저 일반 클릭 시도
                dropdown_element.click()
                print("드롭다운 클릭 성공 (일반 클릭)")
                time.sleep(1)
                return True
            except:
                try:
                    # JavaScript 클릭 시도
                    self.driver.execute_script("arguments[0].click();", dropdown_element)
                    print("드롭다운 클릭 성공 (JavaScript 클릭)")
                    time.sleep(1)
                    return True
                except:
                    try:
                        # ActionChains 클릭 시도
                        actions = ActionChains(self.driver)
                        actions.move_to_element(dropdown_element).click().perform()
                        print("드롭다운 클릭 성공 (ActionChains 클릭)")
                        time.sleep(1)
                        return True
                    except:
                        # 키보드로 스페이스나 엔터 눌러서 열기 시도
                        try:
                            dropdown_element.send_keys(Keys.SPACE)
                            print("드롭다운 열기 성공 (스페이스키)")
                            time.sleep(1)
                            return True
                        except:
                            print("드롭다운 클릭 실패 - 모든 방법 시도함")
                            return False
            
        except Exception as e:
            print(f"드롭다운 클릭 중 오류: {e}")
            return False
            
    def get_dropdown_options(self):
        """드롭다운 옵션들 가져오기 (클릭 후)"""
        try:
            # 먼저 드롭다운을 클릭해서 열기
            if not self.click_dropdown_to_open():
                print("드롭다운을 열 수 없습니다.")
                return []
            
            # 드롭다운 요소 다시 찾기
            dropdown_selectors = [
                "select[name='selectBoxGameRnd']",
                "#selectBoxGameRnd",
                ".selectBoxGameRnd",
                "select:first-of-type"
            ]
            
            dropdown_element = None
            for selector in dropdown_selectors:
                try:
                    dropdown_element = self.driver.find_element(By.CSS_SELECTOR, selector)
                    break
                except:
                    continue
            
            if not dropdown_element:
                print("드롭다운 요소를 다시 찾을 수 없습니다.")
                return []
            
            # 옵션이 로드될 때까지 잠시 대기
            time.sleep(1)
            
            # Select 객체로 옵션 가져오기
            try:
                select = Select(dropdown_element)
                
                # 모든 옵션 가져오기
                options = []
                for option in select.options:
                    option_value = option.get_attribute('value')
                    option_text = option.text.strip()
                    
                    # 빈 값이나 기본값 제외
                    if option_value and option_value != '' and option_text and option_text != '선택':
                        options.append({
                            'value': option_value,
                            'text': option_text
                        })
                
                print(f"드롭다운 옵션 {len(options)}개 발견")
                
                # 옵션 목록 출력 (디버깅용)
                for i, opt in enumerate(options[:5]):  # 처음 5개만 출력
                    print(f"  옵션 {i+1}: {opt['text']} (값: {opt['value']})")
                
                # 디버깅용: 처음 3개만 반환
                if len(options) > 3:
                    print(f"디버깅 모드: 처음 3개 옵션만 처리합니다.")
                    return options[:3]
                
                return options
                
            except Exception as e:
                print(f"Select 객체 생성 또는 옵션 추출 실패: {e}")
                
                # Select가 안 되는 경우 직접 option 태그들 찾기
                try:
                    option_elements = dropdown_element.find_elements(By.TAG_NAME, "option")
                    options = []
                    
                    for option in option_elements:
                        option_value = option.get_attribute('value')
                        option_text = option.text.strip()
                        
                        if option_value and option_value != '' and option_text and option_text != '선택':
                            options.append({
                                'value': option_value,
                                'text': option_text
                            })
                    
                    print(f"직접 추출로 옵션 {len(options)}개 발견")
                    return options[:3] if len(options) > 3 else options
                    
                except Exception as e2:
                    print(f"직접 옵션 추출도 실패: {e2}")
                    return []
            
        except (TimeoutException, NoSuchElementException) as e:
            print(f"드롭다운 처리 실패: {e}")
            return []
    
    def select_dropdown_option(self, option_value):
        """드롭다운에서 특정 옵션 선택"""
        try:
            # 드롭다운 요소 찾기
            dropdown_selectors = [
                "select[name='selectBoxGameRnd']",
                "#selectBoxGameRnd", 
                ".selectBoxGameRnd",
                "select:first-of-type"
            ]
            
            dropdown_element = None
            for selector in dropdown_selectors:
                try:
                    dropdown_element = self.driver.find_element(By.CSS_SELECTOR, selector)
                    break
                except:
                    continue
            
            if not dropdown_element:
                print("드롭다운 요소를 찾을 수 없습니다.")
                return False
            
            # 드롭다운 위치로 스크롤
            self.driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", dropdown_element)
            time.sleep(0.5)
            
            # 먼저 드롭다운 클릭해서 열기
            try:
                dropdown_element.click()
                time.sleep(0.5)
            except:
                self.driver.execute_script("arguments[0].click();", dropdown_element)
                time.sleep(0.5)
            
            # Select 객체로 옵션 선택
            try:
                select = Select(dropdown_element)
                select.select_by_value(option_value)
                print(f"회차 선택: {option_value}")
                time.sleep(1)
                return True
            except Exception as e:
                print(f"Select 옵션 선택 실패: {e}")
                
                # JavaScript로 직접 선택 시도
                try:
                    script = f"""
                    var select = arguments[0];
                    select.value = '{option_value}';
                    select.dispatchEvent(new Event('change'));
                    """
                    self.driver.execute_script(script, dropdown_element)
                    print(f"JavaScript로 회차 선택: {option_value}")
                    time.sleep(1)
                    return True
                except Exception as e2:
                    print(f"JavaScript 선택도 실패: {e2}")
                    return False
            
        except Exception as e:
            print(f"옵션 선택 실패: {e}")
            return False
    
    def click_initial_search_button(self):
        """첫 검색 버튼 클릭 (체크박스가 나타나도록)"""
        try:
            # 검색 버튼 찾기
            search_btn_selectors = [
                "#btn_gmBuySlipSrchDtl",
                "button#btn_gmBuySlipSrchDtl",
                "button.btn.btnM.schTextIcon",
                ".btn.btnM.schTextIcon",
                "button[onclick*='search']"
            ]
            
            search_btn = None
            used_selector = ""
            
            for selector in search_btn_selectors:
                try:
                    search_btn = self.driver.find_element(By.CSS_SELECTOR, selector)
                    used_selector = selector
                    print(f"검색 버튼 발견: {selector}")
                    break
                except:
                    continue
            
            if search_btn:
                # 검색 버튼으로 스크롤
                self.driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", search_btn)
                time.sleep(1)
                
                # JavaScript로 클릭
                self.driver.execute_script("arguments[0].click();", search_btn)
                print(f"첫 검색 버튼 클릭 성공 ({used_selector})")
                
                # 체크박스 로딩 대기
                time.sleep(3)
                return True
            else:
                print("검색 버튼을 찾을 수 없습니다.")
                return False
                
        except Exception as e:
            print(f"첫 검색 버튼 클릭 실패: {e}")
            return False
    
    def set_search_conditions(self):
        """검색 조건 설정 (게임유형: 일반, 대회: KBO)"""
        try:
            # 체크박스가 로드될 때까지 대기
            time.sleep(2)
            
            # 게임유형에서 '일반' 체크박스 선택
            normal_selected = False
            try:
                normal_selectors = [
                    "//input[@type='checkbox' and following-sibling::text()[normalize-space()='일반']]",
                    "//input[@type='checkbox' and @value='일반']",
                    "//label[contains(text(),'일반')]/input[@type='checkbox']",
                    "//label[contains(text(),'일반')]/preceding-sibling::input[@type='checkbox']",
                    "//td[contains(text(),'일반')]/preceding-sibling::td/input[@type='checkbox']"
                ]
                
                for selector in normal_selectors:
                    try:
                        normal_checkbox = self.driver.find_element(By.XPATH, selector)
                        if not normal_checkbox.is_selected():
                            self.driver.execute_script("arguments[0].click();", normal_checkbox)
                            print("게임유형 '일반' 선택")
                            normal_selected = True
                            time.sleep(0.5)
                            break
                    except:
                        continue
                        
                if not normal_selected:
                    print("게임유형 '일반' 체크박스를 찾을 수 없습니다.")
                    
            except Exception as e:
                print(f"일반 체크박스 선택 중 오류: {e}")
            
            # 대회에서 'KBO' 체크박스 선택
            kbo_selected = False
            try:
                kbo_selectors = [
                    "//input[@type='checkbox' and following-sibling::text()[normalize-space()='KBO']]",
                    "//input[@type='checkbox' and @value='KBO']",
                    "//label[contains(text(),'KBO')]/input[@type='checkbox']",
                    "//label[contains(text(),'KBO')]/preceding-sibling::input[@type='checkbox']",
                    "//td[contains(text(),'KBO')]/preceding-sibling::td/input[@type='checkbox']"
                ]
                
                for selector in kbo_selectors:
                    try:
                        kbo_checkbox = self.driver.find_element(By.XPATH, selector)
                        if not kbo_checkbox.is_selected():
                            self.driver.execute_script("arguments[0].click();", kbo_checkbox)
                            print("대회 'KBO' 선택")
                            kbo_selected = True
                            time.sleep(0.5)
                            break
                    except:
                        continue
                        
                if not kbo_selected:
                    print("대회 'KBO' 체크박스를 찾을 수 없습니다.")
                    
            except Exception as e:
                print(f"KBO 체크박스 선택 중 오류: {e}")
            
            return normal_selected or kbo_selected
            
        except Exception as e:
            print(f"검색 조건 설정 중 오류: {e}")
            return False
    
    def click_search_button(self):
        """최종 검색 버튼 클릭 (조건 설정 후)"""
        try:
            search_btn_selectors = [
                "button[id*='slipSrch']",
                "button.btn.btnSch",
                "button[class*='btnSch']",
                "#btn_gmBuySlipSrch",
                ".btn.btnS.schTextIcon",
                "input[value='검색'][type='button']",
                "button[onclick*='search']"
            ]
            
            search_btn = None
            used_selector = ""
            
            for selector in search_btn_selectors:
                try:
                    search_btn = self.driver.find_element(By.CSS_SELECTOR, selector)
                    used_selector = selector
                    print(f"최종 검색 버튼 발견: {selector}")
                    break
                except:
                    continue
            
            # 텍스트 기반으로 검색 버튼 찾기
            if not search_btn:
                try:
                    search_btn = self.driver.find_element(By.XPATH, "//button[contains(text(), '검색')]")
                    used_selector = "text-based search"
                    print("텍스트 기반으로 검색 버튼 발견")
                except:
                    pass
            
            # 모든 버튼을 확인해서 '검색' 텍스트가 있는 버튼 찾기
            if not search_btn:
                try:
                    buttons = self.driver.find_elements(By.TAG_NAME, "button")
                    for btn in buttons:
                        if '검색' in btn.text and '조회' not in btn.text:
                            search_btn = btn
                            used_selector = "text-based search"
                            print("텍스트 기반으로 검색 버튼 발견")
                            break
                except:
                    pass
            
            if search_btn:
                # 검색 버튼으로 스크롤
                self.driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", search_btn)
                time.sleep(1)
                
                # JavaScript로 클릭
                self.driver.execute_script("arguments[0].click();", search_btn)
                print(f"최종 검색 버튼 클릭 완료 ({used_selector})")
                
                # 결과 로딩 대기
                time.sleep(3)
                return True
            else:
                print("최종 검색 버튼을 찾을 수 없습니다.")
                
                # 디버깅을 위해 모든 버튼 정보 출력
                try:
                    buttons = self.driver.find_elements(By.TAG_NAME, "button")
                    print(f"페이지의 모든 버튼 ({len(buttons)}개):")
                    for i, btn in enumerate(buttons):
                        try:
                            btn_text = btn.text.strip()
                            btn_id = btn.get_attribute('id') or '없음'
                            btn_class = btn.get_attribute('class') or '없음'
                            if btn_text:
                                print(f"  버튼 {i+1}: '{btn_text}' (ID: {btn_id}, Class: {btn_class})")
                        except:
                            continue
                except:
                    pass
                return False
                
        except Exception as e:
            print(f"최종 검색 버튼 클릭 실패: {e}")
            return False
    
    def extract_game_info(self):
        """경기 정보 추출"""
        games = []
        
        try:
            # 경기가 없는 경우 체크
            no_data_messages = [
                "조회된 데이터가 없습니다",
                "데이터가 없습니다",
                "경기가 없습니다",
                "해당하는 경기가 없습니다"
            ]
            
            page_text = self.driver.page_source
            for message in no_data_messages:
                if message in page_text:
                    print("해당 조건의 경기가 없습니다. 다음 회차로 넘어갑니다.")
                    return []
            
            # 발매기간 정보 추출
            try:
                sale_period_elem = self.driver.find_element(By.XPATH, "//td[contains(text(), '발매기간')]/following-sibling::td")
                sale_period = sale_period_elem.text.strip() if sale_period_elem else ""
            except:
                sale_period = ""
            
            # 적중결과 발표일 정보 추출
            try:
                result_date_elem = self.driver.find_element(By.XPATH, "//td[contains(text(), '적중결과')]/following-sibling::td")
                result_date = result_date_elem.text.strip() if result_date_elem else ""
            except:
                result_date = ""
            
            # 환급기간 정보 추출
            try:
                refund_period_elem = self.driver.find_element(By.XPATH, "//td[contains(text(), '환급')]/following-sibling::td")
                refund_period = refund_period_elem.text.strip() if refund_period_elem else ""
            except:
                refund_period = ""
            
            # 경기 목록 테이블에서 경기 정보 추출
            try:
                game_tables = self.driver.find_elements(By.CSS_SELECTOR, "table")
                
                for table in game_tables:
                    try:
                        rows = table.find_elements(By.TAG_NAME, "tr")
                        for row in rows:
                            try:
                                cells = row.find_elements(By.TAG_NAME, "td")
                                if len(cells) >= 3:
                                    game_info = {
                                        'sale_period': sale_period,
                                        'result_date': result_date,
                                        'refund_period': refund_period,
                                        'game_data': [cell.text.strip() for cell in cells],
                                        'search_conditions': {
                                            'game_type': '일반',
                                            'league': 'KBO'
                                        }
                                    }
                                    games.append(game_info)
                            except Exception:
                                continue
                    except Exception:
                        continue
                        
            except Exception as e:
                print(f"경기 정보 추출 중 오류: {e}")
            
            # KBO 관련 경기만 필터링
            kbo_games = []
            for game in games:
                game_text = ' '.join(game.get('game_data', []))
                if 'KBO' in game_text or '야구' in game_text or any(team in game_text for team in 
                    ['두산', 'LG', '키움', 'KT', 'SSG', '롯데', 'NC', '삼성', '한화', 'KIA']):
                    kbo_games.append(game)
            
            if kbo_games:
                print(f"KBO 경기 {len(kbo_games)}개 발견")
                return kbo_games
            elif games:
                print(f"일반 경기 {len(games)}개 발견 (KBO 필터링 후)")
                return games
            else:
                print("조건에 맞는 경기가 없습니다.")
                return []
            
        except Exception as e:
            print(f"게임 정보 추출 실패: {e}")
            return []
    
    def crawl_all_options(self):
        """모든 드롭다운 옵션에 대해 크롤링 수행"""
        try:
            # 초기 페이지 스크린샷 저장
            self.driver.save_screenshot("initial_page.png")
            
            # 드롭다운 옵션들 가져오기 (클릭해서 열기 포함)
            options = self.get_dropdown_options()
            
            if not options:
                print("드롭다운 옵션을 찾을 수 없습니다.")
                return
            
            print(f"발견된 옵션들: {[opt['text'] for opt in options]}")
            print(f"디버깅 모드: {len(options)}개 옵션 처리 예정")
            
            # 각 옵션별로 크롤링
            for i, option in enumerate(options):
                print(f"\n=== 옵션 {i+1}/{len(options)}: {option['text']} ===")
                
                # 페이지 새로고침 (상태 초기화)
                if i > 0:
                    self.driver.refresh()
                    time.sleep(2)
                
                # 1단계: 옵션 선택
                if not self.select_dropdown_option(option['value']):
                    print("옵션 선택 실패")
                    continue
                
                # 2단계: 첫 검색 버튼 클릭 (체크박스 나타나게 하기)
                if not self.click_initial_search_button():
                    print("첫 검색 버튼 클릭 실패")
                    continue
                
                # 3단계: 검색 조건 설정 (일반, KBO 체크박스 선택)
                if not self.set_search_conditions():
                    print("검색 조건 설정 실패")
                
                # 4단계: 최종 검색 버튼 클릭
                if not self.click_search_button():
                    print("최종 검색 버튼 클릭 실패")
                    continue
                
                # 결과 페이지 스크린샷
                self.driver.save_screenshot(f"result_{i+1}.png")
                
                # 5단계: 경기 정보 추출
                games = self.extract_game_info()
                
                if games:
                    # 결과에 추가
                    round_data = {
                        'round_info': option,
                        'games': games,
                        'search_conditions': {
                            'game_type': '일반',
                            'league': 'KBO'
                        },
                        'crawl_time': time.strftime('%Y-%m-%d %H:%M:%S')
                    }
                    
                    self.results.append(round_data)
                    print(f"KBO 일반 경기 {len(games)}개 추출 완료")
                else:
                    print("해당 회차에는 KBO 일반 경기가 없습니다. 다음 회차로 넘어갑니다.")
                
                # 다음 옵션 처리 전 잠시 대기
                time.sleep(2)
                    
        except Exception as e:
            print(f"크롤링 중 오류 발생: {e}")
            self.driver.save_screenshot("error_screenshot.png")
    
    def save_results(self, filename='betman_games.json'):
        """결과를 JSON 파일로 저장"""
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(self.results, f, ensure_ascii=False, indent=2)
            print(f"결과가 {filename}에 저장되었습니다.")
            
            # Excel 파일로도 저장
            excel_filename = filename.replace('.json', '.xlsx')
            self.save_to_excel(excel_filename)
            
        except Exception as e:
            print(f"파일 저장 실패: {e}")
    
    def save_to_excel(self, filename='betman_games.xlsx'):
        """결과를 Excel 파일로 저장"""
        try:
            all_games = []
            
            for round_data in self.results:
                round_info = round_data['round_info']['text']
                for game in round_data['games']:
                    game_row = {
                        'round': round_info,
                        'sale_period': game.get('sale_period', ''),
                        'result_date': game.get('result_date', ''),
                        'refund_period': game.get('refund_period', ''),
                        'crawl_time': round_data['crawl_time']
                    }
                    
                    # 게임 데이터 추가
                    if 'game_data' in game:
                        for j, data in enumerate(game['game_data']):
                            game_row[f'column_{j+1}'] = data
                    
                    all_games.append(game_row)
            
            if all_games:
                df = pd.DataFrame(all_games)
                df.to_excel(filename, index=False, engine='openpyxl')
                print(f"Excel 파일이 {filename}에 저장되었습니다.")
            
        except Exception as e:
            print(f"Excel 저장 실패: {e}")
    
    def run(self):
        """크롤러 실행"""
        try:
            print("베트맨 크롤러 시작 (개선된 속도 + 정확한 검색 버튼)")
            
            # 드라이버 설정
            self.setup_driver()
            
            # 페이지 이동
            self.navigate_to_page()
            
            # 모든 옵션 크롤링 (3개로 제한)
            self.crawl_all_options()
            
            # 결과 저장
            if self.results:
                self.save_results()
                print(f"\n총 {len(self.results)}개 라운드의 경기 정보를 수집했습니다.")
            else:
                print("수집된 데이터가 없습니다.")
                
        except Exception as e:
            print(f"크롤러 실행 중 오류: {e}")
        
        finally:
            if self.driver:
                self.driver.quit()
                print("브라우저 종료")

def main():
    """메인 함수"""
    crawler = BetmanCrawler()
    crawler.run()

if __name__ == "__main__":
    # 필요한 패키지 설치 안내
    print("필요한 패키지들:")
    print("pip install selenium webdriver-manager pandas openpyxl")
    print("\n개선된 크롤러를 시작합니다...\n")
    
    main()

필요한 패키지들:
pip install selenium webdriver-manager pandas openpyxl

개선된 크롤러를 시작합니다...

베트맨 크롤러 시작 (개선된 속도 + 정확한 검색 버튼)
크롬 드라이버 설정 완료
페이지 로딩 중: https://www.betman.co.kr/main/mainPage/gamebuy/closedGameSlipIFR.do?gmId=G101&gmTs=250053&gameDivCd=C&isIFR=Y
페이지 로딩 완료
드롭다운 발견: #selectBoxGameRnd
드롭다운 클릭 성공 (일반 클릭)
드롭다운 옵션 340개 발견
  옵션 1: 62회차 [05월26일 월요일 회차마감] (값: G101,250062)
  옵션 2: 61회차 [05월23일 금요일 회차마감] (값: G101,250061)
  옵션 3: 60회차 [05월21일 수요일 회차마감] (값: G101,250060)
  옵션 4: 59회차 [05월19일 월요일 회차마감] (값: G101,250059)
  옵션 5: 58회차 [05월16일 금요일 회차마감] (값: G101,250058)
디버깅 모드: 처음 3개 옵션만 처리합니다.
발견된 옵션들: ['62회차 [05월26일 월요일 회차마감]', '61회차 [05월23일 금요일 회차마감]', '60회차 [05월21일 수요일 회차마감]']
디버깅 모드: 3개 옵션 처리 예정

=== 옵션 1/3: 62회차 [05월26일 월요일 회차마감] ===
회차 선택: G101,250062
검색 버튼 발견: #btn_gmBuySlipSrchDtl
첫 검색 버튼 클릭 성공 (#btn_gmBuySlipSrchDtl)
게임유형 '일반' 선택
대회 'KBO' 선택
최종 검색 버튼 발견: button.btn.btnSch
최종 검색 버튼 클릭 완료 (button.btn.btnSch)
KBO 경기 20개 발견
KBO 일반 경기 20개 추출 완료

=== 옵션 2/3: 61회차 [05월23일 금요일 회차마감] ===
회차 선택: G10